# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing
import os
import math

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
}

def get_content(url):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url).content
    return r

url = 'https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv?encoding=UTF-8'

file_path = 'data/data-20160608T1215-structure-20160608T1215.csv'

if not os.path.isfile(file_path):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    s = get_content(url)

    with open(file_path, 'wb') as f:
        f.write(s)

# df = pd.read_csv(io.StringIO(s.decode('UTF8')))
df = pd.read_csv(file_path)

### Если не работает загрузка on-line
#df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')

display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [3]:
# Ваше решение

df = pd.read_csv(file_path)

# Шаблон для float значения.
_float_pattern = r'\d*\.?\d+'

def to_float(float_str):
    """
    Конвертирование строки в число с типом float.
    """
    try:
        return float(''.join(re.findall(_float_pattern, float_str.replace(',', '.'))))
    except ValueError:
        pass
    return 0.0


def check(value, standard):
    """
    Проверка значения value на соответствие нормативу standard.
    """
    value = to_float(value)
    standard = standard.replace(',', '.')

    # Норматив в формате 'в пределах min-max'.
    m = re.match(rf'в пределах ({_float_pattern})-({_float_pattern})', standard)
    if m:
        min = to_float(m.group(1))
        max = to_float(m.group(2))
        return 'Да' if min <= value <= max else 'Нет'

    # Норматив в формате 'не более max'.
    m = re.match(rf'не более ({_float_pattern})', standard)
    if m:
        max = to_float(m.group(1))
        return 'Да' if value < max else 'Нет'

    # По умолчанию не соответствует нормативу.
    return 'Нет'

df['Результат проверки'] = df.apply(lambda _df: check(_df['Результат анализа'], _df['Норматив']), axis=1)

display(df)

,Показатель,Единица измерений,Результат анализа,Норматив,Результат проверки
0,pH,единицы pH,8.4,в пределах 6-9,Да
1,Запах,баллы,1,не более 2-3,Да
2,Цветность,градусы,б/цвета,не более 30,Да
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Да
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",Да
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Да
6,Нитраты (по NO3),мг/дм3,24,не более 45,Да
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5",Да
8,Хлориды (Cl),мг/дм3,200,не более 350,Да
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,Да


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?

**Решение:**

Дано:
- $M = 3$ - количество выбираемых фруктов
- $N = 9$ - общее количество фруктов
- $N_a = 5$ - количество апельсинов
- $N_b = 4$ - количество блок

Общая формула, которая позволяет найти число сочетаний из $n$ объектов по $k$ имеет вид  
$C_n^k = \frac{n!}{(n-k)!\cdot k!}$

Число случаев вынуть 3 апельсина  
$C(A) = C_{N_a}^{M} = C_5^3 = 10$  

Общее число случаев вынуть 3 фрукта  
$C(B) = C_{N}^{M} = C_9^3 = 84$  

Тогда искомая вероятность будет иметь значение  
$P = \frac{C(A)}{C(B)} = \frac{C_5^3}{C_9^3} = \frac{10}{84} = 0.119047$

In [4]:
# Ваше решение

n = 3 # Количество выбираемых фруктов
n_orange = 5 # Количество апельсинов
n_apple = 4 # Количество яблок

fruit_quantity = n_orange + n_apple # Общее количество фруктов

# Заполняем ящик фруктами
fruit_box = np.array(['orange'] * n_orange + ['apple'] * n_apple)

# Ожидаемое (расчетное) значение
prob_expected = math.comb(n_orange, n) / math.comb(n_orange + n_apple, n)

# Вычисляем статистику для различного числа экспериментов
simulations = [10, 100, 1000, 10000, 100000]

for n_simulation in simulations:
    prob = 0

    for i in range(n_simulation):
        # Перед экспериментом перемешиваем фрукты в ящике
        np.random.shuffle(fruit_box)
        # Выбрать 3-и случайных фрукта
        sample = np.random.choice(range(fruit_quantity), size=n, replace=False)

        if all(['orange' == fruit_box[x] for x in sample]):
            prob += 1

    prob /= n_simulation

    print(f'probability={prob:0.3f}\tprob_expected={prob_expected:0.3f}\tn_simulation={n_simulation}')

probability=0.200	prob_expected=0.119	n_simulation=10
probability=0.060	prob_expected=0.119	n_simulation=100
probability=0.128	prob_expected=0.119	n_simulation=1000
probability=0.122	prob_expected=0.119	n_simulation=10000
probability=0.119	prob_expected=0.119	n_simulation=100000


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

**Решение:**  

Дано:
- $N = 10$ - общее количество деталей
- $N_a = 3$ - количество нестандартных деталей
- $N_b = 7$ - количество стандартных деталей

Вероятность того, что при первой проверке попадется **нестандартная** деталь  
$P(A) = \frac{N_a}{N}$

Условная вероятность того, что при второй проверке попадется **стандартная** деталь при условии, что первая детать **нестандартная**  
$P(B|A) = \frac{N_b}{N-1}$

Вероятность того, что мастер проверит ровно две детали  
$P = P(AB) = P(A) \cdot P(B|A)$

Тогда по теореме умножения вероятностей искомая вероятность будет иметь значение  
$P = P(AB) = P(A) \cdot P(B|A) = \frac{3}{10} \cdot \frac{7}{9} = \frac{21}{90} = \frac{7}{30} = 0.233333$

In [5]:
# Ваше решение

n = 10 # Количество деталей
n_nonstandard = 3 # Количество нестандартных деталей
n_standard = n - n_nonstandard # Количество стандартных деталей

# Детали для проверки
details = list(['nonstandard'] * n_nonstandard + ['standard'] * n_standard)

prob_expected = 0.23333

def check_detail(details):
    n_details = len(details) # Количество деталей
    n_sample = 0 # Номер по счету проверяемой детали

    for i in reversed(range(n)):
        # Взять случайную деталь
        sample = np.random.choice(range(n_details - i), size=1, replace=False)[0]
        detail = details.pop(sample)

        # Если деталь стандартная, то ...
        if 'standard' == detail:
            # ... какая это деталь по счету
            if n_sample == 1:
                # Было проверено 2-е детали
                return True
            break

        n_sample += 1
                
    return False

# Вычисляем статистику для различного числа экспериментов
simulations = [10, 100, 1000, 10000, 100000]

for i, n_simulation in enumerate(simulations):
    prob = 0

    for j in range(n_simulation):
        # Подготовка данных перед экспериментом
        details_copy = details.copy()
        np.random.shuffle(details_copy)

        if check_detail(details_copy):
            prob += 1

    prob /= n_simulation

    print(f'probability={prob:0.3f}\tprob_expected={prob_expected:0.3f}\tn_simulation={n_simulation}')

probability=0.300	prob_expected=0.233	n_simulation=10
probability=0.310	prob_expected=0.233	n_simulation=100
probability=0.243	prob_expected=0.233	n_simulation=1000
probability=0.238	prob_expected=0.233	n_simulation=10000
probability=0.231	prob_expected=0.233	n_simulation=100000
